# 🦅 AVCI - Yüksek Oran Avcısı Eğitim Paneli v6.1.0 (Detailed & Explicit)
**Güncelleme Notları:**
- **🔍 Şeffaf Eğitim:** Optuna kodları ve ayarları artık hücrelerin içinde açıkça görünüyor. İstediğin gibi müdahale edebilirsin.
- **⚙️ Tam Kontrol:** Her hedef (1.5x, 3.0x...) için deneme sayısını (EPOCHS) ve parametreleri hücre içinde değiştirebilirsin.
- **Veri Kurtarıcı (Data Savior):** Drive yedeklemesi aktif.
---

In [ ]:
# 1. GitHub Deposunu Çek ve Kurulum Yap (Temiz Kurulum)
import os
import shutil

# 1. Kök dizine dön
try:
    os.chdir('/content')
except:
    pass

# 2. Varsa eski projeyi SİL (Her şeyi sıfırla ki eski hatalar kalmasın)
if os.path.exists('newproje'):
    print("Eski proje dosyaları temizleniyor...")
    shutil.rmtree('newproje')

# 3. Sıfırdan İndir
print("Projeyi GitHub'dan sıfırdan çekiyorum...")
!git clone https://github.com/onndd/newproje.git

# 4. Çalışma dizinine gir
if os.path.exists('newproje/avci'):
    os.chdir('newproje/avci')
    print(f"✅ Kurulum Başarılı. Çalışma Dizini: {os.getcwd()}")
    
    # --- VERİTABANI KURTARMA ---
    drive_db_path = '/content/drive/MyDrive/newproje/avci/jetx.db'
    local_db_path = 'jetx.db'
    
    if not os.path.exists('/content/drive'):
        try:
            from google.colab import drive
            drive.mount('/content/drive')
        except:
            pass

    if os.path.exists(drive_db_path):
        print(f"🔄 Gerçek Veritabanı Drive'dan kopyalanıyor... ({os.path.getsize(drive_db_path)/1024/1024:.2f} MB)")
        shutil.copy(drive_db_path, local_db_path)
        print("✅ Veritabanı güncellendi.")
    else:
        print("⚠️ UYARI: Drive'da 'jetx.db' bulunamadı! Lütfen dosyanın yerinde olduğundan emin ol.")
else:
    print("❌ HATA: İndirme başarısız oldu!")

print("GPU Durumu:")
!nvidia-smi

# Kütüphane Kurulumları
!pip install optuna streamlit matplotlib pandas tensorflow hmmlearn plotly lightgbm
print("Kurulum ve Hazırlık Tamamlandı! Artık modüller bulunabilir.")

In [ ]:
# 2. Veri Yükleme ve Ayarlar
import optuna
import logging
import sys
import os

optuna.logging.set_verbosity(optuna.logging.INFO)
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))

try:
    # Modülleri içe aktar (Absolute Import)
    import train_avci
    import models_avci
    import model_ae_avci
    import model_hmm_avci
    
    # Fonksiyonları kısayol olarak al
    load_and_prep = train_avci.load_and_prep
    visualize_performance = train_avci.visualize_performance
    get_scoring_params = train_avci.get_scoring_params
    
    print("✅ Modüller başarıyla yüklendi!")
except ImportError as e:
    print(f"❌ HATA: Modül import hatası. Detay: {e}")

if 'load_and_prep' in locals():
    try:
        df = load_and_prep(limit=100000)
        print(f"Veri Hazır: {len(df)} satır. (Sütunlar: {len(df.columns)})")
    except Exception as e:
         print(f"❌ VERİ YÜKLEME HATASI: {e}")

## 🕵️‍♂️ Bölüm 1: İstihbarat & Komuta

In [ ]:
# Autoencoder (Anomali Dedektörü)
if 'model_ae_avci' in locals():
    ae = model_ae_avci.train_autoencoder(df)

In [ ]:
# HMM (Piyasa Rejimi)
if 'model_hmm_avci' in locals():
    hmm_model = model_hmm_avci.train_hmm(df)

## 🎯 Bölüm 2: Nişancı Eğitimi (Detaylı Optuna)

In [ ]:
# --- EĞİTİM AYARLARI ---
USE_GPU = True
features = [c for c in df.columns if 'target' not in c and 'result' not in c and 'value' not in c and 'id' not in c]
X = df[features]

# Veri Setini Böl (Son %15 Test)
split_idx = int(len(df) * 0.85)
X_train, X_val = X.iloc[:split_idx], X.iloc[split_idx:]

In [ ]:
# 🎯 HEDEF 1.50x EĞİTİMİ
TARGET = 1.5
EPOCHS = 30

print(f"\n🚀 {TARGET}x Hedefi İçin Optuna Başlatılıyor...")
y_col = f'target_{str(TARGET).replace(".","_")}'
y_train, y_val = df[y_col].iloc[:split_idx], df[y_col].iloc[split_idx:]
scoring = get_scoring_params(TARGET)

# Optuna Study
study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: models_avci.objective_lgbm(trial, X_train, y_train, X_val, y_val, scoring, USE_GPU), n_trials=EPOCHS)

print(f"En İyi Skor: {study.best_value}")
print(f"En İyi Parametreler: {study.best_params}")

# Final Model Eğitimi
final_params = study.best_params
final_params.update({'metric': 'binary_logloss', 'objective': 'binary', 'verbosity': -1})
if USE_GPU: 
    final_params.update({'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0})

model = models_avci.train_lgbm(X_train, y_train, X_val, y_val, final_params)
model.save_model(f'models/avci_lgbm_{str(TARGET).replace(".","_")}.txt')

# Performans Analizi
visualize_performance(model, X_val, y_val, TARGET)
try: optuna.visualization.plot_optimization_history(study).show()
except: pass

In [ ]:
# 🎯 HEDEF 3.00x EĞİTİMİ
TARGET = 3.0
EPOCHS = 30

print(f"\n🚀 {TARGET}x Hedefi İçin Optuna Başlatılıyor...")
y_col = f'target_{str(TARGET).replace(".","_")}'
y_train, y_val = df[y_col].iloc[:split_idx], df[y_col].iloc[split_idx:]
scoring = get_scoring_params(TARGET)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: models_avci.objective_lgbm(trial, X_train, y_train, X_val, y_val, scoring, USE_GPU), n_trials=EPOCHS)

final_params = study.best_params
final_params.update({'metric': 'binary_logloss', 'objective': 'binary', 'verbosity': -1})
if USE_GPU: final_params.update({'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0})

model = models_avci.train_lgbm(X_train, y_train, X_val, y_val, final_params)
model.save_model(f'models/avci_lgbm_{str(TARGET).replace(".","_")}.txt')
visualize_performance(model, X_val, y_val, TARGET)
try: optuna.visualization.plot_optimization_history(study).show()
except: pass

In [ ]:
# 🎯 HEDEF 5.00x EĞİTİMİ
TARGET = 5.0
EPOCHS = 50

print(f"\n🚀 {TARGET}x Hedefi İçin Optuna Başlatılıyor...")
y_col = f'target_{str(TARGET).replace(".","_")}'
y_train, y_val = df[y_col].iloc[:split_idx], df[y_col].iloc[split_idx:]
scoring = get_scoring_params(TARGET)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: models_avci.objective_lgbm(trial, X_train, y_train, X_val, y_val, scoring, USE_GPU), n_trials=EPOCHS)

final_params = study.best_params
final_params.update({'metric': 'binary_logloss', 'objective': 'binary', 'verbosity': -1})
if USE_GPU: final_params.update({'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0})

model = models_avci.train_lgbm(X_train, y_train, X_val, y_val, final_params)
model.save_model(f'models/avci_lgbm_{str(TARGET).replace(".","_")}.txt')
visualize_performance(model, X_val, y_val, TARGET)
try: optuna.visualization.plot_optimization_history(study).show()
except: pass

In [ ]:
# 🎯 HEDEF 10.0x EĞİTİMİ
TARGET = 10.0
EPOCHS = 50

print(f"\n🚀 {TARGET}x Hedefi İçin Optuna Başlatılıyor...")
y_col = f'target_{str(TARGET).replace(".","_")}'
y_train, y_val = df[y_col].iloc[:split_idx], df[y_col].iloc[split_idx:]
scoring = get_scoring_params(TARGET)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: models_avci.objective_lgbm(trial, X_train, y_train, X_val, y_val, scoring, USE_GPU), n_trials=EPOCHS)

final_params = study.best_params
final_params.update({'metric': 'binary_logloss', 'objective': 'binary', 'verbosity': -1})
if USE_GPU: final_params.update({'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0})

model = models_avci.train_lgbm(X_train, y_train, X_val, y_val, final_params)
model.save_model(f'models/avci_lgbm_{str(TARGET).replace(".","_")}.txt')
visualize_performance(model, X_val, y_val, TARGET)
try: optuna.visualization.plot_optimization_history(study).show()
except: pass

In [ ]:
# 🎯 HEDEF 100.0x EĞİTİMİ (Avcı Modu)
TARGET = 100.0
EPOCHS = 100

print(f"\n🚀 {TARGET}x Hedefi İçin Optuna Başlatılıyor...")
y_col = f'target_{str(TARGET).replace(".","_")}'
y_train, y_val = df[y_col].iloc[:split_idx], df[y_col].iloc[split_idx:]
scoring = get_scoring_params(TARGET)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: models_avci.objective_lgbm(trial, X_train, y_train, X_val, y_val, scoring, USE_GPU), n_trials=EPOCHS)

final_params = study.best_params
final_params.update({'metric': 'binary_logloss', 'objective': 'binary', 'verbosity': -1})
if USE_GPU: final_params.update({'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0})

model = models_avci.train_lgbm(X_train, y_train, X_val, y_val, final_params)
model.save_model(f'models/avci_lgbm_{str(TARGET).replace(".","_")}.txt')
visualize_performance(model, X_val, y_val, TARGET)
try: optuna.visualization.plot_optimization_history(study).show()
except: pass

In [ ]:
# GOD MODE: 1000.0x
TARGET = 1000.0
EPOCHS = 150

print(f"\n🚀 {TARGET}x Hedefi İçin Optuna Başlatılıyor...")
y_col = f'target_{str(TARGET).replace(".","_")}'
y_train, y_val = df[y_col].iloc[:split_idx], df[y_col].iloc[split_idx:]
scoring = get_scoring_params(TARGET)

study = optuna.create_study(direction='maximize')
study.optimize(lambda trial: models_avci.objective_lgbm(trial, X_train, y_train, X_val, y_val, scoring, USE_GPU), n_trials=EPOCHS)

final_params = study.best_params
final_params.update({'metric': 'binary_logloss', 'objective': 'binary', 'verbosity': -1})
if USE_GPU: final_params.update({'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0})

model = models_avci.train_lgbm(X_train, y_train, X_val, y_val, final_params)
model.save_model(f'models/avci_lgbm_{str(TARGET).replace(".","_")}.txt')
visualize_performance(model, X_val, y_val, TARGET)
try: optuna.visualization.plot_optimization_history(study).show()
except: pass

## 💾 Bölüm 3: Modelleri Kaydet ve İndir

In [ ]:
# Tüm Modelleri Ziple
import os
if not os.path.exists('models'):
    os.makedirs('models')
    
!zip -r avci_models.zip models/

# İndir
try:
    from google.colab import files
    files.download('avci_models.zip')
    print("Dosya indiriliyor... 'avci_models.zip'")
except Exception as e:
    print(f"İndirme hatası: {e}. Sol taraftaki dosya gezgininden 'avci_models.zip' dosyasını sağ tıklayıp indirebilirsin.")

# Varsa Drive'a Kopyala
if os.path.exists('/content/drive/MyDrive'):
    !cp avci_models.zip /content/drive/MyDrive/avci_backup_latest.zip
    print("Yedek Drive'a (Ana Dizin) kopyalandı.")